In [7]:
import tensorflow as tf

In [8]:
import h5py as h5

In [9]:
#tf.reset_default_graph()


modelDir='knee_trained_MoDL'

loadChkPoint=tf.train.latest_checkpoint(modelDir)

In [14]:
with h5.File('../train/file_brain_AXFLAIR_200_6002425.h5','r') as f:
    print(f.keys())
    print(f['reconstruction_rss'])
    print(f['kspace'])

<KeysViewHDF5 ['ismrmrd_header', 'kspace', 'reconstruction_rss']>
<HDF5 dataset "reconstruction_rss": shape (16, 320, 320), type "<f4">
<HDF5 dataset "kspace": shape (16, 20, 640, 320), type "<c8">


In [15]:
with h5.File('knee_demo_data.h5','r') as f:
    print(f.keys())
    print(f['atb'])

<KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
<HDF5 dataset "atb": shape (1, 640, 368), type "<c8">


In [ ]:
with h5.File('demoImage.hdf5','r') as f:
    print(f.keys())
    print(f['tstMask'])